# 🧪 02_train_effnet_kaggle — CSIRO Image2Biomass (Production Baseline)

**Goal:** Train EfficientNet‑B0 + metadata fusion regressor with portable, identical behavior across macOS (MPS), CPU, and Kaggle GPU — within the 9‑hour limit.

**Rules:**
- Uses only the verified repository context; **no internet**.
- Imports from `src/` when present; **does not modify** any `src/` files.
- Provides safe in‑notebook fallbacks if a given helper is missing.
- Saves artifacts under `output/checkpoints`, `output/logs`, `output/submissions`.


In [1]:
pip install -q albumentations==1.4.7

Note: you may need to restart the kernel to use updated packages.


In [2]:
# =============================================================
# 🔧 Cell 1 — Setup: Paths, Imports, and Environment (kernel-safe)
# =============================================================
import os
import sys
import math
import time
import random
import json
import shutil
from pathlib import Path
import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# -----------------------------
# 🔹 Robust PROJECT_ROOT detection
# -----------------------------
PROJECT_ROOT = Path.cwd()
if (PROJECT_ROOT / "src").exists():
    pass
elif PROJECT_ROOT.name == "notebooks" and (PROJECT_ROOT.parent / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
else:
    PR = PROJECT_ROOT
    for _ in range(3):
        if (PR / "src").exists():
            PROJECT_ROOT = PR
            break
SRC = PROJECT_ROOT / "src"

# -----------------------------
# 🔹 Standard input/output paths
# -----------------------------
INPUT_LOCAL = PROJECT_ROOT / "input_local"
OUTPUT_DIR = PROJECT_ROOT / "output"
CKPT_DIR = OUTPUT_DIR / "checkpoints"
LOG_DIR = OUTPUT_DIR / "logs"
SUBM_DIR = OUTPUT_DIR / "submissions"

for d in [OUTPUT_DIR, CKPT_DIR, LOG_DIR, SUBM_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# -----------------------------
# 🔹 Kernel-safe sys.path patch
# Add PROJECT_ROOT to sys.path so that src.* imports work
# -----------------------------
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"📂 Project root: {PROJECT_ROOT}")
print(f"📦 Using src path: {SRC}")

# -----------------------------
# 🔹 Verify core src files exist
# -----------------------------
required_files = [
    "config.py", "env.py", "data_loading.py", "data_pipeline.py",
    "model_utils.py", "train_utils.py", "inference_utils.py", "feature_engineering.py"
]
available = {f: (SRC / f).exists() for f in required_files}
print("🗂️ Verified src files:")
for k, v in available.items():
    print(f"  - {k}: {'✅' if v else '❌'}")


📂 Project root: /Users/olia_/projects/Kaggle/csiro-biomass
📦 Using src path: /Users/olia_/projects/Kaggle/csiro-biomass/src
🗂️ Verified src files:
  - config.py: ✅
  - env.py: ✅
  - data_loading.py: ✅
  - data_pipeline.py: ✅
  - model_utils.py: ✅
  - train_utils.py: ✅
  - inference_utils.py: ✅
  - feature_engineering.py: ✅


In [3]:
# =============================================================
# 🌍 Cell 2 — Device & Worker Configuration (kernel-safe)
# =============================================================
import torch

try:
    # Attempt to use src.env get_env()
    from src.env import get_env
    DEVICE, NUM_WORKERS, PIN_MEMORY = get_env()
    print("✅ get_env() from src/env.py")
except Exception as e:
    # Fallback hardware detection (local deterministic)
    if torch.cuda.is_available():
        DEVICE = "cuda"
        NUM_WORKERS = 2
        PIN_MEMORY = True
    else:
        # Force CPU fallback, ignore MPS for clarity
        DEVICE = "cpu"
        NUM_WORKERS = 0
        PIN_MEMORY = False
    print(f"⚠️ Fallback env applied: DEVICE={DEVICE}, NUM_WORKERS={NUM_WORKERS}, PIN_MEMORY={PIN_MEMORY} | reason: {e}")

print(f"🧭 Using device: {DEVICE}")
print(f"👷 DataLoader workers: {NUM_WORKERS} | pin_memory: {PIN_MEMORY}")


🧭 Device: mps | num_workers=0 | pin_memory=False
✅ get_env() from src/env.py
🧭 Using device: mps
👷 DataLoader workers: 0 | pin_memory: False


In [4]:
# =============================================================
# 🎲 Cell 3 — Reproducibility: Deterministic Seeds
# =============================================================
SEED = 42

# -----------------------------
# 🔹 Python & NumPy seeds
# -----------------------------
import random
import numpy as np
import torch

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# -----------------------------
# 🔹 Device-specific seeds
# -----------------------------
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)
elif DEVICE == "mps":
    # MPS backend seed handling (if needed)
    torch.manual_seed(SEED)

# -----------------------------
# 🔹 Deterministic backend behavior
# -----------------------------
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"✅ Seeds set for deterministic runs on {DEVICE}")


✅ Seeds set for deterministic runs on mps


In [5]:
# =============================================================
# 🧩 Pivot train.csv for multi-output image regression (../input_local/)
# =============================================================
import pandas as pd
from pathlib import Path

# --- Correct simple path ---
train_path = Path("../input_local/train.csv")  # 👈 one level up from notebooks

if not train_path.exists():
    raise FileNotFoundError(f"❌ Could not find {train_path.resolve()}")

# --- Load original training data ---
df = pd.read_csv(train_path)
print(f"📂 Loaded train.csv → shape: {df.shape}")

# --- Pivot so each image_path appears once with 5 target columns ---
pivot_df = (
    df.pivot_table(
        index="image_path",
        columns="target_name",
        values="target",
        aggfunc="first"
    )
    .reset_index()
)

# --- Drop incomplete rows if any ---
pivot_df = pivot_df.dropna(subset=["Dry_Total_g"])
print(f"✅ Pivoted dataset shape: {pivot_df.shape}")

# --- Save pivoted file beside train.csv ---
pivot_path = train_path.parent / "train_pivoted.csv"
pivot_df.to_csv(pivot_path, index=False)
print(f"💾 Saved pivoted training data → {pivot_path}")

# --- Sanity checks ---
unique_images = df["image_path"].nunique()
print(f"🧮 Unique images before pivot: {unique_images}")
assert unique_images == len(pivot_df), "Pivot mismatch — check for missing targets"

print("✅ Pivot complete. Ready for image-only multi-output training.")


📂 Loaded train.csv → shape: (1785, 9)
✅ Pivoted dataset shape: (357, 6)
💾 Saved pivoted training data → ../input_local/train_pivoted.csv
🧮 Unique images before pivot: 357
✅ Pivot complete. Ready for image-only multi-output training.


In [6]:
# =============================================================
# 📦 Cell 4 — Build Datasets and DataLoaders (DINOv2-Compatible, 518×518)
# =============================================================
from pathlib import Path
import sys
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

# --- Try Albumentations; fallback to torchvision if unavailable ---
try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    USE_ALB = True
    print("✅ Using Albumentations for augmentations.")
except ImportError:
    from torchvision import transforms
    USE_ALB = False
    print("⚠️ Albumentations not found — falling back to torchvision transforms.")

# -----------------------------
# 🔹 Paths
# -----------------------------
DATA_ROOT = Path("../input_local")
TRAIN_CSV = DATA_ROOT / "train_pivoted.csv"
TEST_CSV  = DATA_ROOT / "test.csv"

if not TRAIN_CSV.exists():
    raise FileNotFoundError(f"❌ Missing: {TRAIN_CSV.resolve()}")
if not TEST_CSV.exists():
    raise FileNotFoundError(f"❌ Missing: {TEST_CSV.resolve()}")

# -----------------------------
# 🔹 Global constants (linked to model)
# -----------------------------
IMG_SIZE = globals().get("IMG_SIZE", 518)  # ✅ match DINOv2 ViT-B/14
USE_LOG_TARGET = True
LOG_EPS = 1.0
BATCH_SIZE = 8 if DEVICE == "mps" else (32 if DEVICE == "cuda" else 8)  # DINOv2 = larger images
NUM_WORKERS = 0

# -----------------------------
# 🔹 Define augmentations (stronger for ViTs)
# -----------------------------
if USE_ALB:
    train_tfms = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE, interpolation=1),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.15, rotate_limit=30, p=0.7),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.4),
        A.HueSaturationValue(p=0.3),
        A.RGBShift(p=0.3),
        A.RandomGamma(p=0.3),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2(),
    ])
    valid_tfms = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE, interpolation=1),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2(),
    ])
else:
    from torchvision import transforms
    train_tfms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])
    valid_tfms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])

# -----------------------------
# 🔹 Dataset class
# -----------------------------
class ImageOnlyDataset(Dataset):
    def __init__(self, csv_path, img_root, transform, train=True, use_log=False):
        self.df = pd.read_csv(csv_path)
        self.img_root = Path(img_root)
        self.transform = transform
        self.train = train
        self.use_log = use_log
        if train:
            self.target_cols = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]

    def __len__(self): 
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_root / row["image_path"]
        image = Image.open(img_path).convert("RGB")

        if USE_ALB:
            image = np.array(image)
            image = self.transform(image=image)["image"]
        else:
            image = self.transform(image)

        if self.train:
            target = row[self.target_cols].values.astype("float32")
            if self.use_log:
                target = np.log1p(target)
            return image, torch.tensor(target, dtype=torch.float32)
        else:
            sample_id = row.get("sample_id", f"test_{idx}")
            return image, sample_id

# -----------------------------
# 🔹 DataLoaders
# -----------------------------
train_ds = ImageOnlyDataset(TRAIN_CSV, img_root=DATA_ROOT, transform=train_tfms, train=True, use_log=USE_LOG_TARGET)
test_ds  = ImageOnlyDataset(TEST_CSV,  img_root=DATA_ROOT, transform=valid_tfms, train=False)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=False)
test_dl  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=False)

print(f"✅ Train dataset: {len(train_ds)} samples | Test dataset: {len(test_ds)} samples")
imgs, targets = next(iter(train_dl))
print(f"🎞️ Batch shapes — images: {imgs.shape}, targets: {targets.shape}")
print(f"✅ Dataloader sanity check passed (IMG_SIZE={IMG_SIZE}, backbone=DINOv2 ViT-B/14).")


✅ Using Albumentations for augmentations.
✅ Train dataset: 357 samples | Test dataset: 5 samples
🎞️ Batch shapes — images: torch.Size([8, 3, 518, 518]), targets: torch.Size([8, 5])
✅ Dataloader sanity check passed (IMG_SIZE=518, backbone=DINOv2 ViT-B/14).


In [7]:
# =============================================================
# 🧠 Cell 5 — Multi-Output Model Definition (DINOv2 ViT-B/14, Stable)
# =============================================================
import torch
import torch.nn as nn
import timm

# -------------------------------------------------------------
# 🔹 DINOv2 model family + image sizes
# -------------------------------------------------------------
IMG_SIZE_MAP = {
    "vit_base_patch14_dinov2.lvd142m": 518,  # true DINOv2 ViT-B/14
    "efficientnet_b3": 300,
}
NUM_OUTPUTS = 5

# -------------------------------------------------------------
# 🔹 Model wrapper
# -------------------------------------------------------------
class ImageRegressor(nn.Module):
    def __init__(self, backbone_name="vit_base_patch14_dinov2.lvd142m", pretrained=True, num_outputs=NUM_OUTPUTS, dropout=0.3):
        super().__init__()
        self.backbone_name = backbone_name
        try:
            self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
            print(f"✅ Loaded backbone: {backbone_name}")
        except Exception as e:
            print(f"⚠️ Could not load {backbone_name} ({e}); using EfficientNet-B3 fallback.")
            self.backbone_name = "efficientnet_b3"
            self.backbone = timm.create_model("efficientnet_b3", pretrained=True, num_classes=0)

        # DINOv2 uses embed_dim, EfficientNet uses num_features
        in_features = getattr(self.backbone, "embed_dim", None) or getattr(self.backbone, "num_features", 1024)
        hidden = 768 if "dinov2" in self.backbone_name else 512
        self.head = nn.Sequential(
            nn.Linear(in_features, hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_outputs),
        )

    def forward(self, x):
        feats = self.backbone(x)
        return self.head(feats)

# -------------------------------------------------------------
# 🔹 Initialize
# -------------------------------------------------------------
BACKBONE = "vit_base_patch14_dinov2.lvd142m"
IMG_SIZE = IMG_SIZE_MAP.get(BACKBONE, 300)
model = ImageRegressor(backbone_name=BACKBONE, pretrained=True, num_outputs=NUM_OUTPUTS).to(DEVICE)
globals()["IMG_SIZE"] = IMG_SIZE

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"🧮 Trainable parameters: {n_params/1e6:.2f} M | Device: {DEVICE}")


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/vit_base_patch14_dinov2.lvd142m)
INFO:timm.models._hub:[timm/vit_base_patch14_dinov2.lvd142m] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


✅ Loaded backbone: vit_base_patch14_dinov2.lvd142m
🧮 Trainable parameters: 87.17 M | Device: mps


In [8]:
# =============================================================
# 🏋️ Cell 6 — 30-Minute Quiet Training (DINOv2 ViT-B/14, Static ETA Bar)
# =============================================================
import os, time, logging, warnings, numpy as np, torch, torch.nn as nn
from pathlib import Path
from tqdm.notebook import tqdm
from src.config import PROJECT_ROOT

# -------------------------------
# 🔕 Silence framework noise
# -------------------------------
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("torch").setLevel(logging.ERROR)

# -------------------------------
# 🔧 Config
# -------------------------------
USE_AMP = (DEVICE == "cuda")
EPOCHS = 120               # ≈ 30 min on MPS / CPU
LR = 2e-5                  # small LR for ViT fine-tuning
WD = 1e-4
WARMUP_EPOCHS = 5
UNFREEZE_AT = 20
PATIENCE = None            # 🚫 disable early-stop completely
PRINT_EVERY = 10

BACKBONE_NAME = globals().get("BACKBONE", "vit_base_patch14_dinov2.lvd142m")
BEST_CKPT = PROJECT_ROOT / "output" / "checkpoints" / f"{BACKBONE_NAME}_best.pth"
BEST_CKPT.parent.mkdir(parents=True, exist_ok=True)
BEST_CKPT = str(BEST_CKPT)

# -------------------------------
# 🔹 Metrics
# -------------------------------
def _inverse_log(x): return np.expm1(x)

def _compute_metrics(y_true, y_pred, use_log=True):
    if use_log:
        y_true, y_pred = _inverse_log(y_true), _inverse_log(y_pred)
    y_true, y_pred = y_true.flatten(), y_pred.flatten()
    rmse = float(np.sqrt(np.mean((y_true - y_pred)**2)))
    mae  = float(np.mean(np.abs(y_true - y_pred)))
    r2   = float(1 - np.sum((y_true - y_pred)**2) /
                 (np.sum((y_true - np.mean(y_true))**2) + 1e-12))
    return dict(rmse=rmse, mae=mae, r2=r2)

# -------------------------------
# 🔹 Optimizer & Scheduler
# -------------------------------
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)
criterion = nn.SmoothL1Loss(beta=0.5)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

def adjust_lr(epoch):
    if epoch <= WARMUP_EPOCHS:
        lr = LR * (epoch / WARMUP_EPOCHS)
        for g in optimizer.param_groups:
            g["lr"] = lr

# -------------------------------
# 🔹 Freeze / Unfreeze
# -------------------------------
for p in model.backbone.parameters():
    p.requires_grad = False  # start frozen

def unfreeze_backbone_layers(model, ratio=0.3):
    layers = list(model.backbone.parameters())
    cutoff = int(len(layers) * (1 - ratio))
    for p in layers[cutoff:]:
        p.requires_grad = True

# -------------------------------
# 🔹 Epoch Routine
# -------------------------------
def run_epoch(dl, train=True):
    model.train() if train else model.eval()
    losses, y_true_all, y_pred_all = [], [], []
    for imgs, targets in dl:
        imgs, targets = imgs.to(DEVICE), targets.to(DEVICE)
        with torch.set_grad_enabled(train):
            with torch.amp.autocast(device_type="cuda" if USE_AMP else "cpu", enabled=USE_AMP):
                preds = model(imgs)
                loss = criterion(preds, targets)
            if train:
                optimizer.zero_grad(set_to_none=True)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
        losses.append(loss.item())
        y_true_all.append(targets.cpu().numpy())
        y_pred_all.append(preds.detach().cpu().numpy())
    y_true, y_pred = np.concatenate(y_true_all), np.concatenate(y_pred_all)
    return np.mean(losses), _compute_metrics(y_true, y_pred, use_log=USE_LOG_TARGET)

# -------------------------------
# 🔹 Main Training Loop (quiet)
# -------------------------------
best_rmse, best_state = float("inf"), None
epoch_times = []

print(f"🚀 {BACKBONE_NAME} | {EPOCHS} epochs | IMG_SIZE={IMG_SIZE} | Device={DEVICE}")
print(f"Samples = {len(train_ds)} | Batch = {train_dl.batch_size}\n")

t0 = time.time()
progress = tqdm(total=EPOCHS, desc="⏳ Training Progress", position=0, leave=True, miniters=1)

for epoch in range(1, EPOCHS + 1):
    adjust_lr(epoch)
    start = time.time()
    tr_loss, tr_metrics = run_epoch(train_dl, train=True)
    scheduler.step()
    epoch_time = time.time() - start
    epoch_times.append(epoch_time)
    progress.update(1)
    eta = (EPOCHS - epoch) * np.mean(epoch_times) / 60

    if epoch == UNFREEZE_AT:
        unfreeze_backbone_layers(model, ratio=0.3)
        print(f"🔓 Unfroze top 30 % of ViT layers at epoch {epoch}")

    if (epoch % PRINT_EVERY == 0) or (epoch in {1, UNFREEZE_AT, EPOCHS}):
        print(f"Epoch {epoch:03d}/{EPOCHS} | RMSE {tr_metrics['rmse']:.2f} | "
              f"MAE {tr_metrics['mae']:.2f} | R² {tr_metrics['r2']:.3f} | "
              f"{epoch_time:.1f}s | ETA {eta:.1f} min")

    if tr_metrics["rmse"] < best_rmse - 1e-5:
        best_rmse = tr_metrics["rmse"]
        best_state = {k: v.cpu() for k, v in model.state_dict().items()}
        torch.save(best_state, BEST_CKPT)

progress.close()
total_time = (time.time() - t0) / 60

# -------------------------------
# 🔹 Finalize
# -------------------------------
if best_state:
    model.load_state_dict(best_state)
    print(f"\n✅ Best checkpoint loaded | RMSE {best_rmse:.2f}")
else:
    print("\n⚠️ No improvement — using final weights.")
print(f"🏁 Total training time {total_time:.1f} min | Backbone {BACKBONE_NAME}")


🚀 vit_base_patch14_dinov2.lvd142m | 120 epochs | IMG_SIZE=518 | Device=mps
Samples = 357 | Batch = 8



⏳ Training Progress:   0%|          | 0/120 [00:00<?, ?it/s]

Epoch 001/120 | RMSE 35.62 | MAE 24.57 | R² -0.903 | 85.8s | ETA 170.3 min
Epoch 010/120 | RMSE 23.61 | MAE 14.40 | R² 0.163 | 83.0s | ETA 155.9 min
🔓 Unfroze top 30 % of ViT layers at epoch 20
Epoch 020/120 | RMSE 18.53 | MAE 11.08 | R² 0.485 | 84.3s | ETA 141.2 min
Epoch 030/120 | RMSE 14.68 | MAE 8.19 | R² 0.677 | 125.3s | ETA 151.5 min
Epoch 040/120 | RMSE 13.23 | MAE 7.61 | R² 0.738 | 131.8s | ETA 144.2 min
Epoch 050/120 | RMSE 11.87 | MAE 6.65 | R² 0.789 | 140.0s | ETA 132.4 min
Epoch 060/120 | RMSE 10.80 | MAE 5.88 | R² 0.825 | 137.5s | ETA 117.1 min
Epoch 070/120 | RMSE 10.05 | MAE 5.57 | R² 0.849 | 127.6s | ETA 99.2 min
Epoch 080/120 | RMSE 11.13 | MAE 6.14 | R² 0.814 | 128.1s | ETA 80.2 min
Epoch 090/120 | RMSE 11.96 | MAE 6.17 | R² 0.785 | 133.0s | ETA 60.8 min
Epoch 100/120 | RMSE 11.10 | MAE 6.01 | R² 0.815 | 133.3s | ETA 40.7 min
Epoch 110/120 | RMSE 9.55 | MAE 5.08 | R² 0.863 | 127.6s | ETA 20.5 min
Epoch 120/120 | RMSE 9.84 | MAE 4.91 | R² 0.855 | 134.1s | ETA 0.0 min



In [11]:
# =============================================================
# 📈 Cell 7 — Final Inference & Kaggle Submission (DINOv2 + Extended TTA)
# =============================================================
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import time

# -------------------------------
# 🔧 Device & model prep
# -------------------------------
device = next(model.parameters()).device
model.eval()

# -------------------------------
# 📂 Paths
# -------------------------------
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = PROJECT_ROOT / "input_local"
OUTPUT_DIR = PROJECT_ROOT / "output" / "submissions"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

test_path = DATA_DIR / "test.csv"
img_root = DATA_DIR
if not test_path.exists():
    raise FileNotFoundError(f"❌ Missing {test_path.resolve()}")

# -------------------------------
# 📑 Load test.csv
# -------------------------------
test_df = pd.read_csv(test_path)
print(f"📂 Loaded test.csv → shape: {test_df.shape}")

# -------------------------------
# 🧠 Build test dataset
# -------------------------------
IMG_SIZE = getattr(train_ds, "img_size", 518)
test_ds = ImageOnlyDataset(
    csv_path=test_path,
    img_root=img_root,
    transform=valid_tfms,
    train=False
)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=0)

# =============================================================
# 🔹 Inference with Extended TTA (flips + 90° rotations)
# =============================================================
def infer_with_tta(imgs):
    """
    Perform inference with flips and 90° rotations.
    DINOv2 ViTs are rotation-tolerant, so averaging these helps generalize.
    """
    variants = [
        imgs,
        torch.flip(imgs, dims=[2]),              # vertical
        torch.flip(imgs, dims=[3]),              # horizontal
        torch.rot90(imgs, 1, [2, 3]),
        torch.rot90(imgs, 2, [2, 3]),
        torch.rot90(imgs, 3, [2, 3]),
    ]
    preds_all = [model(v) for v in variants]
    return torch.stack(preds_all).mean(dim=0)

# =============================================================
# 🔹 Run Inference Loop
# =============================================================
all_preds = []
start = time.time()

with torch.no_grad():
    for imgs, ids in test_dl:
        imgs = imgs.to(device, non_blocking=True)
        preds = infer_with_tta(imgs)
        preds = preds.cpu().numpy()
        if USE_LOG_TARGET:
            preds = np.expm1(preds)  # inverse log transform
        all_preds.append(preds)

preds = np.concatenate(all_preds, axis=0)
print(f"✅ Inference complete → predictions shape: {preds.shape}")
print(f"🕒 Inference time: {(time.time() - start):.1f}s")

# =============================================================
# 🔹 Format Submission
# =============================================================
target_cols = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]
unique_images = test_df["image_path"].unique()

if len(unique_images) != preds.shape[0]:
    print(f"⚠️ Adjusting predictions: {preds.shape[0]} rows vs {len(unique_images)} images")
    unique_images = np.resize(unique_images, preds.shape[0])

preds_df = pd.DataFrame(preds, columns=target_cols)
preds_df["image_path"] = unique_images

sub_df = preds_df.melt(
    id_vars="image_path", var_name="target_name", value_name="target"
)
final_sub = (
    test_df[["sample_id", "image_path", "target_name"]]
    .merge(sub_df, on=["image_path", "target_name"], how="left")
    .drop_duplicates(subset=["sample_id"])
)

# =============================================================
# 💾 Save & Report
# =============================================================
sub_path = OUTPUT_DIR / "submission.csv"
final_sub[["sample_id", "target"]].to_csv(sub_path, index=False)

print(f"✅ Saved submission → {sub_path.resolve()}")
print(f"📦 Submission shape: {final_sub.shape}")
print(final_sub["target"].describe().to_string())
print("🏁 Ready for Kaggle upload.")


📂 Loaded test.csv → shape: (5, 3)
✅ Inference complete → predictions shape: (5, 5)
🕒 Inference time: 19.6s
⚠️ Adjusting predictions: 5 rows vs 1 images
✅ Saved submission → /Users/olia_/projects/Kaggle/csiro-biomass/output/submissions/submission.csv
📦 Submission shape: (5, 4)
count     5.000000
mean     26.450842
std      18.799717
min       0.005757
25%      20.874638
50%      27.795448
75%      31.519236
max      52.059128
🏁 Ready for Kaggle upload.
